# فهرست مطالب
- [درباره Guidance](../../../../code/01.Introduce)
- [راه‌اندازی](../../../../code/01.Introduce)
- [تولید بدون محدودیت](../../../../code/01.Introduce)
- [صحبت کردن برای Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [انتخاب](../../../../code/01.Introduce)
- [زنجیره تفکر](../../../../code/01.Introduce)
- [تولید JSON](../../../../code/01.Introduce)
- [تولید HTML](../../../../code/01.Introduce)


# درباره Guidance
Guidance یک کتابخانه متن‌باز اثبات‌شده در زبان پایتون است که برای کنترل خروجی‌های هر مدل زبانی (LM) طراحی شده است. با یک فراخوانی API، می‌توانید محدودیت‌های برنامه‌ریزی دقیق را (در پایتون) بیان کنید که مدل باید از آن‌ها پیروی کند و خروجی ساختاریافته را در قالب JSON، Python، HTML، SQL یا هر ساختاری که مورد نیاز باشد، تولید کنید.

Guidance با تکنیک‌های معمولی درخواست‌دهی متفاوت است. این ابزار محدودیت‌ها را با هدایت مدل به صورت توکن به توکن در لایه استنتاج اعمال می‌کند، که منجر به تولید خروجی‌های با کیفیت بالاتر می‌شود و هزینه و تأخیر را تا ۳۰–۵۰٪ در سناریوهای بسیار ساختاریافته کاهش می‌دهد.

برای اطلاعات بیشتر درباره Guidance، به [مخزن عمومی در GitHub](https://github.com/guidance-ai/guidance) مراجعه کنید یا [جلسه Breakout Guidance](https://www.youtube.com/watch?v=qXMNPVVlCMs) را در Microsoft Build مشاهده کنید.


# تنظیمات
1. Guidance را با دستور `pip install guidance --pre` نصب کنید
2. یک نقطه پایانی Phi 3.5 mini را در Azure مستقر کنید. برای این کار به آدرس https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml بروید و روی دکمه "Deploy" کلیک کنید
3. کلید API نقطه پایانی خود را در یک متغیر محیطی به نام `AZURE_PHI3_KEY` ذخیره کنید و URL آن را در یک متغیر محیطی به نام `AZURE_PHI3_URL` قرار دهید


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# تولید بدون محدودیت
متن می‌تواند بدون هیچ محدودیتی با استفاده از تابع `gen()` تولید شود. این همان استفاده از مدل بدون Guidance است.

## قالب‌بندی چت
مانند بسیاری از مدل‌های چت، Phi-3 انتظار دارد پیام‌ها بین کاربر و دستیار در قالب خاصی باشند. Guidance از قالب چت Phi-3 پشتیبانی می‌کند و قالب‌بندی چت را برای شما مدیریت خواهد کرد. برای ایجاد نوبت‌های چت، هر بخش از مکالمه را در یک بلوک `with user()` یا `with assistant()` قرار دهید. یک بلوک `with system()` می‌تواند برای تنظیم پیام سیستم استفاده شود.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## صرفه‌جویی در توکن‌ها
در سناریوهای بسیار ساختاریافته، Guidance می‌تواند توکن‌ها را نادیده بگیرد و فقط توکن‌های ضروری را تولید کند، که این امر عملکرد را بهبود می‌بخشد، کارایی را افزایش می‌دهد و هزینه‌های API را کاهش می‌دهد. توکن‌های تولید شده در این دفترچه با یک پس‌زمینه برجسته نشان داده شده‌اند. توکن‌های اجباری بدون برجسته‌سازی نمایش داده می‌شوند و هزینه‌ای مشابه با توکن‌های ورودی دارند، که تقریباً یک سوم هزینه توکن‌های خروجی تخمین زده می‌شود.

*توجه:* اولین مثال با تولید بدون محدودیت قادر به اجبار هیچ توکنی نبود زیرا هیچ محدودیتی ارائه نشده بود.


# صحبت درباره Phi 3  
با استفاده از Guidance، می‌توانید به راحتی متن را در پاسخ‌های مدل وارد کنید. این کار می‌تواند مفید باشد اگر بخواهید خروجی مدل را در یک جهت خاص هدایت کنید.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# محدود کردن با استفاده از regex  
در مثال قبلی، Phi 3 پس از پاسخ دادن به سؤال با `Canberra` توضیحات بیشتری ارائه داد. برای محدود کردن خروجی مدل به دقیقاً یک کلمه، می‌توان از regex استفاده کرد.  


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

با استفاده از عبارت منظم، فقط کلمه `کانبرا` تولید می‌شود.


# انتخاب از میان چند گزینه  
وقتی برخی گزینه‌های ممکن مشخص هستند، می‌توانید از تابع `select()` استفاده کنید تا مدل از میان لیستی از گزینه‌ها انتخاب کند.  


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

با `select()`، تنها نشانه `Can` تولید شد. چون `Canberra` تنها گزینه‌ای است که می‌تواند پاسخ را کامل کند، نشانه‌های باقی‌مانده مجبور شدند.


# زنجیره تفکر  
زنجیره تفکر تکنیکی است که می‌تواند به بهبود کیفیت خروجی مدل کمک کند، با تشویق آن به پردازش مسئله به صورت مرحله به مرحله. معمولاً برای رسیدن به پاسخ نهایی، چندین مرحله درخواست لازم است. ابتدا مدل را راهنمایی کنید تا مرحله به مرحله فکر کند. سپس دوباره از مدل بخواهید که پاسخ نهایی را ارائه دهد. با استفاده از API‌های استاندارد استنتاج چت، این فرآیند نیاز به دو تماس API دارد و زنجیره تفکر تولید شده توسط مدل دوبار هزینه‌بردار می‌شود – یک بار به عنوان توکن‌های خروجی زمانی که مدل آن را تولید می‌کند، و بار دیگر به عنوان توکن‌های ورودی برای تماس دوم. با استفاده از Guidance، کل فرآیند چندمرحله‌ای به عنوان بخشی از یک تماس API پردازش و هزینه‌بردار می‌شود، که هزینه و تأخیر را کاهش می‌دهد.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# تولید JSON
می‌توان از Guidance برای تضمین تولید JSON مطابق با یک JSON schema یا مدل پایدانتیک استفاده کرد، مانند schema پروفایل کاربر که در اینجا نشان داده شده است.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## تولید HTML

راهنما می‌تواند برای تولید کد و رعایت الزامات نحوی در زبان برنامه‌نویسی استفاده شود. در این بخش، یک برنامه کوچک راهنما برای نوشتن صفحات وب HTML بسیار ساده ایجاد خواهیم کرد.

صفحه وب را به بخش‌های کوچکتر تقسیم می‌کنیم، هر کدام با یک تابع راهنما مخصوص به خود. سپس این بخش‌ها را در تابع نهایی خود ترکیب می‌کنیم تا یک صفحه وب HTML ایجاد کنیم.  
در نهایت، این تابع را در برابر یک مدل فعال‌شده با راهنما در Azure AI اجرا خواهیم کرد.

[!NOTE] این یک تولیدکننده HTML کامل نخواهد بود؛ هدف این است که نشان دهیم چگونه می‌توانید خروجی ساختاریافته‌ای برای نیازهای خاص خود ایجاد کنید.

ابتدا با وارد کردن موارد مورد نیاز از راهنما شروع می‌کنیم:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

صفحات وب HTML بسیار ساختارمند هستند و ما بخش‌های مختلف صفحه را با استفاده از Guidance "اجبار" خواهیم کرد.  
وقتی به طور مشخص از مدل متن درخواست می‌کنیم، باید مطمئن شویم که هیچ چیزی که ممکن است به عنوان یک تگ شناخته شود، شامل نمی‌شود - به این معنا که باید کاراکترهای '<' و '>' را حذف کنیم:


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

سپس می‌توانیم از این تابع برای تولید متن در یک تگ HTML دلخواه استفاده کنیم:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

حالا بیایید عنوان صفحه را ایجاد کنیم.  
به عنوان بخشی از این کار، باید یک عنوان صفحه تولید کنیم:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

بدنه صفحه HTML قرار است با عناوین و پاراگراف‌ها پر شود.  
ما می‌توانیم یک تابع برای انجام هر کدام تعریف کنیم:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

اکنون، تابعی برای تعریف بدنه خود HTML.

این تابع از `select()` با `recurse=True` استفاده می‌کند تا چندین عنوان و پاراگراف ایجاد کند:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

سپس به تابعی می‌رسیم که صفحه کامل HTML را تولید می‌کند.  
ابتدا تگ شروع HTML را اضافه می‌کنیم، سپس هدر را تولید می‌کنیم، بعد بدنه را ایجاد می‌کنیم و در نهایت تگ پایانی HTML را اضافه می‌کنیم:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

ما یک پوشش کاربرپسند ارائه می‌دهیم که به ما امکان می‌دهد:
- دمای تولید را تنظیم کنیم
- صفحه تولید شده را از شیء مدل ضبط کنیم


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

سپس می‌توانیم خروجی را در یک فایل بنویسیم:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

و [نتیجه را ببینید](../../../../code/01.Introduce/sample_page.html).



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
